# Imports


In [15]:
!pip install transformers torch # take this out before submitting project
!pip install openai==0.28
from transformers import BertModel, BertTokenizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import string
import torch
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [16]:
def get_sbert_embedding(word):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embedding = model.encode(word)
    return embedding

In [17]:
#Generating a dataset of embeddings from a list of words

list_of_words = ["move", "rotate", "draw", "change_colors", "pen_down"]
df_sbert = pd.DataFrame(columns = ["verb", "embedding"])
for word in list_of_words:
  df_sbert = pd.concat([df_sbert, pd.DataFrame([{"verb": word, "embedding": get_sbert_embedding(word)}])], ignore_index=True)
df_sbert

,verb,embedding
0,move,"[0.024703356, -0.03555954, 0.026363244, -0.028..."
1,rotate,"[-0.0047539584, 0.054445975, -0.058048636, -0...."
2,draw,"[0.02108034, 0.02142658, -0.027628534, -0.0167..."
3,change_colors,"[0.0036541086, 0.03610057, 0.018009296, -0.015..."
4,pen_down,"[0.00167139, 0.03985135, 0.011079031, 0.013109..."


In [18]:

def compare_verb_to_df (verb, df = df_sbert):
  #We start off by generating an embedding using Bert for our verb :
  embedding_verb = get_sbert_embedding(verb)

  # Reshape embedding_verb to be a 2D array
  embedding_verb = embedding_verb.reshape(1, -1)  # Reshape to (1, 768)

  similarity = 0
  final_verb = ""
  for iter,row in df.iterrows():
    new_verb, new_embe = row['verb'], row['embedding']

    # Reshape new_embe to be a 2D array
    new_embe = new_embe.reshape(1, -1)  # Reshape to (1, 768) if new_embe is (768,)

    # Calculate cosine similarity correctly
    new_similarity = cosine_similarity(embedding_verb, new_embe)
    if new_similarity > similarity:
      similarity = new_similarity
      final_verb = new_verb
  return final_verb

result = compare_verb_to_df("travel")
result

'move'

In [19]:
def get_sbert_embedding(word):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embedding = model.encode(word)
    return embedding

In [20]:
#Generating a dataset of embeddings from a list of words

list_of_words = ["move", "rotate", "draw", "change_colors", "pen_down"]
df_sbert = pd.DataFrame(columns = ["verb", "embedding"])
for word in list_of_words:
  df_sbert = pd.concat([df_sbert, pd.DataFrame([{"verb": word, "embedding": get_sbert_embedding(word)}])], ignore_index=True)
df_sbert

,verb,embedding
0,move,"[0.024703356, -0.03555954, 0.026363244, -0.028..."
1,rotate,"[-0.0047539584, 0.054445975, -0.058048636, -0...."
2,draw,"[0.02108034, 0.02142658, -0.027628534, -0.0167..."
3,change_colors,"[0.0036541086, 0.03610057, 0.018009296, -0.015..."
4,pen_down,"[0.00167139, 0.03985135, 0.011079031, 0.013109..."


In [21]:

def compare_verb_to_df (verb, df = df_sbert):
  #We start off by generating an embedding using Bert for our verb :
  embedding_verb = get_sbert_embedding(verb)

  # Reshape embedding_verb to be a 2D array
  embedding_verb = embedding_verb.reshape(1, -1)  # Reshape to (1, 768)

  similarity = 0
  final_verb = ""
  for iter,row in df.iterrows():
    new_verb, new_embe = row['verb'], row['embedding']

    # Reshape new_embe to be a 2D array
    new_embe = new_embe.reshape(1, -1)  # Reshape to (1, 768) if new_embe is (768,)

    # Calculate cosine similarity correctly
    new_similarity = cosine_similarity(embedding_verb, new_embe)
    if new_similarity > similarity:
      similarity = new_similarity
      final_verb = new_verb
  return final_verb

result = compare_verb_to_df("travel")
result

'move'

In [22]:
import string

def extract_actions(tree, df=df_sbert):
    actions = []
    direction_counter = {}  # To track the count of DIRECTION for each action type (e.g., move)
    distance_counter = {}  # To track the count of DISTANCE for each action type (e.g., move)

    for subtree in tree.subtrees():
        if subtree.label() == 'ACTION':
            action_data = {}
            verb = subtree.leaves()[0]
            final_verb = compare_verb_to_df(verb, df)  # Check verb against the DataFrame
            action_data["action_type"] = final_verb

            # Track the occurrence of each action type (e.g., move)
            if final_verb == 'move':
                # Count the number of directions and distances associated with 'move'
                if final_verb not in direction_counter:
                    direction_counter[final_verb] = 0
                direction_counter[final_verb] += 1

                if final_verb not in distance_counter:
                    distance_counter[final_verb] = 0
                distance_counter[final_verb] += 1

                # Extract direction and distance based on their counts for each 'move'
                directions = [leaf for subtree in tree.subtrees() if subtree.label() == 'DIRECTION' for leaf in subtree.leaves()]
                direction = directions[direction_counter[final_verb] - 1] if direction_counter[final_verb] <= len(directions) else 'left'

                distances = [leaf for subtree in tree.subtrees() if subtree.label() == 'DISTANCE' for leaf in subtree.leaves()]
                distance = distances[distance_counter[final_verb] - 1] if distance_counter[final_verb] <= len(distances) else '1'

                # Extract other properties for 'move'
                lin_velocity = next(
                    (leaf for subtree in tree.subtrees() if subtree.label() == 'VELOCITY' for leaf in subtree.leaves()), '10'
                )

                action_data['direction'] = direction
                action_data['distance'] = distance
                action_data['lin_velocity'] = lin_velocity

            elif final_verb == 'rotate':
                orientation = next(
                    (leaf for subtree in tree.subtrees() if subtree.label() == 'ORIENTATION' for leaf in subtree.leaves()), 'clockwise'
                )
                angle = next(
                    (leaf for subtree in tree.subtrees() if subtree.label() == 'ANGLE' for leaf in subtree.leaves()), '30'
                )
                action_data['orientation'] = orientation
                action_data['angle'] = angle

            elif final_verb == 'draw':
                figure = next(
                    (leaf for subtree in tree.subtrees() if subtree.label() == 'FIGURE' for leaf in subtree.leaves()), 'triangle'
                )
                action_data['figure'] = figure

            elif final_verb == 'change_colors':
                color = next(
                    (leaf for subtree in tree.subtrees() if subtree.label() == 'COLOR' for leaf in subtree.leaves()), 'red'
                )
                action_data['color'] = color

            elif final_verb == 'pen_down':
                action_data['pen_down'] = True
            else:
                action_data['pen_down'] = False  # For cases where no action type is matched

            actions.append(action_data)

    return actions




def preprocess_command(command, df):
    """
    Preprocesses the command by:
    1. Converting to lowercase.
    2. Removing punctuation except when between two potential digits.
    3. Resolving each verb to its final_verb using the given dataframe.
    """
    import re

    # Convert to lowercase
    command = command.lower()

    # Use regex to remove punctuation except when between digits
    # Pattern explanation:
    # (?<!\d) ensures the character before the punctuation is NOT a digit
    # (?!\d) ensures the character after the punctuation is NOT a digit
    command = re.sub(r"(?<!\d)[.,!?;:](?!\d)", "", command)

    # Process the entire command with spaCy
    doc = nlp(command)

    # Iterate through tokens to resolve verbs
    resolved_tokens = []
    for token in doc:
        # print(f"Token: {token}, POS: {token.pos_}")  # Debugging output
        if token.pos_ == "VERB":
            try:
                # Resolve verb to final_verb
                final_verb = compare_verb_to_df(token.text, df)
                resolved_tokens.append(final_verb)
            except Exception:
                # If no resolution is found, keep the original token
                resolved_tokens.append(token.text)
        else:
            # Include relevant non-verb tokens
            resolved_tokens.append(token.text)

    # Reconstruct the resolved command
    resolved_command = " ".join(resolved_tokens)
    #print(f'this is the resolved command: <{resolved_command}>')
    return resolved_command




In [23]:
angles = " | ".join(f"'{i}'" for i in range(1, 361))
print(angles)

'1' | '2' | '3' | '4' | '5' | '6' | '7' | '8' | '9' | '10' | '11' | '12' | '13' | '14' | '15' | '16' | '17' | '18' | '19' | '20' | '21' | '22' | '23' | '24' | '25' | '26' | '27' | '28' | '29' | '30' | '31' | '32' | '33' | '34' | '35' | '36' | '37' | '38' | '39' | '40' | '41' | '42' | '43' | '44' | '45' | '46' | '47' | '48' | '49' | '50' | '51' | '52' | '53' | '54' | '55' | '56' | '57' | '58' | '59' | '60' | '61' | '62' | '63' | '64' | '65' | '66' | '67' | '68' | '69' | '70' | '71' | '72' | '73' | '74' | '75' | '76' | '77' | '78' | '79' | '80' | '81' | '82' | '83' | '84' | '85' | '86' | '87' | '88' | '89' | '90' | '91' | '92' | '93' | '94' | '95' | '96' | '97' | '98' | '99' | '100' | '101' | '102' | '103' | '104' | '105' | '106' | '107' | '108' | '109' | '110' | '111' | '112' | '113' | '114' | '115' | '116' | '117' | '118' | '119' | '120' | '121' | '122' | '123' | '124' | '125' | '126' | '127' | '128' | '129' | '130' | '131' | '132' | '133' | '134' | '135' | '136' | '137' | '138' | '139

In [24]:
# Generate numbers with 0.1 increments
decimal_digits = [f"'{i / 10:.1f}'" for i in range(1, 1000)]  # '0.1' to '99.9'
decimal_digits.append("'100'")  # Add '100' for the last value

# Generate whole numbers
whole_numbers = [f"'{i}'" for i in range(1, 101)]  # '1' to '100'

# Combine both lists and join with " | "
all_digits = " | ".join(decimal_digits + whole_numbers)
print(all_digits)

'0.1' | '0.2' | '0.3' | '0.4' | '0.5' | '0.6' | '0.7' | '0.8' | '0.9' | '1.0' | '1.1' | '1.2' | '1.3' | '1.4' | '1.5' | '1.6' | '1.7' | '1.8' | '1.9' | '2.0' | '2.1' | '2.2' | '2.3' | '2.4' | '2.5' | '2.6' | '2.7' | '2.8' | '2.9' | '3.0' | '3.1' | '3.2' | '3.3' | '3.4' | '3.5' | '3.6' | '3.7' | '3.8' | '3.9' | '4.0' | '4.1' | '4.2' | '4.3' | '4.4' | '4.5' | '4.6' | '4.7' | '4.8' | '4.9' | '5.0' | '5.1' | '5.2' | '5.3' | '5.4' | '5.5' | '5.6' | '5.7' | '5.8' | '5.9' | '6.0' | '6.1' | '6.2' | '6.3' | '6.4' | '6.5' | '6.6' | '6.7' | '6.8' | '6.9' | '7.0' | '7.1' | '7.2' | '7.3' | '7.4' | '7.5' | '7.6' | '7.7' | '7.8' | '7.9' | '8.0' | '8.1' | '8.2' | '8.3' | '8.4' | '8.5' | '8.6' | '8.7' | '8.8' | '8.9' | '9.0' | '9.1' | '9.2' | '9.3' | '9.4' | '9.5' | '9.6' | '9.7' | '9.8' | '9.9' | '10.0' | '10.1' | '10.2' | '10.3' | '10.4' | '10.5' | '10.6' | '10.7' | '10.8' | '10.9' | '11.0' | '11.1' | '11.2' | '11.3' | '11.4' | '11.5' | '11.6' | '11.7' | '11.8' | '11.9' | '12.0' | '12.1' | '12.2' | '

# Rule Based Method Approach:

In [25]:
import spacy
from nltk import CFG
from nltk.parse import ChartParser

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Define the updated CFG grammar
base_grammar = f"""
S -> ACTION OBJECT DIRECTION ADVERB DIRECTION ADVERB ACTION ORIENTATION
S -> ACTION OBJECT DIRECTION ADVERB DIRECTION ADVERB ACTION ORIENTATION CONJ S
S -> ACTION OBJECT DIRECTION ADVERB ACTION ORIENTATION
S -> ACTION OBJECT DIRECTION ADVERB ACTION ORIENTATION CONJ S
S -> ACTION OBJECT PREP DIRECTION ADVERB
S -> ACTION OBJECT PREP LOCATION
S -> ACTION ADVERB OBJECT PREP DIRECTION
S -> ACTION OBJECT CONJ ACTION OBJECT
S -> ACTION OBJECT
S -> ACTION OBJECT PREP PREP DIRECTION
S -> ACTION OBJECT PREP PREP DIRECTION DISTANCE
S -> ACTION OBJECT PREP DIRECTION
S -> ACTION OBJECT PREP DIRECTION VELOCITY
S -> ACTION OBJECT PREP TOOL
S -> ACTION OBJECT PREP OBJECT
S -> ACTION OBJECT ORIENTATION
S -> ACTION OBJECT PREP OBJECT PREP LOCATION
S -> ACTION OBJECT CONJ ACTION OBJECT PREP LOCATION
S -> ACTION OBJECT PREP WORD
S -> ACTION OBJECT CONJ ACTION OBJECT PREP LOCATION
S -> ACTION OBJECT DIRECTION ADVERB ACTION DIRECTION ADVERB ACTION ORIENTATION
S -> ACTION PREP COLOR
S -> ACTION OBJECT PREP COLOR
S -> ACTION CONJ FIGURE
S -> ACTION | ACTION S
S -> ACTION OBJECT ITEM DIRECTION NUMBER UNIT
S -> ACTION PREP PREP DIRECTION DISTANCE CONJ DISTANCE PREP ANGLE UNITS ORIENTATION
S -> ACTION PREP PREP DIRECTION DISTANCE ADVERB ACTION DISTANCE COUNTERCLOCKWISE
S -> ACTION OBJECT ORIENTATION ADVERB ACTION DIRECTION
S -> ACTION ITEM PREP COLOR
S -> ACTION PREP DIRECTION ADVERB ACTION PREP DIRECTION ADVERB ACTION ORIENTATION
S -> ACTION DRAWING ADVERB ACTION DRAWING
S -> ACTION DRAWING ADVERB ACTION DIRECTION
S -> ACTION OBJECT ITEM PREP COLOR CONJ ACTION DRAWING
S -> ACTION OBJECT CONJ ACTION DIRECTION
S -> PEN_DOWN OBJECT CONJ ACTION DIRECTION NUMBER CONJ NUMBER
S -> ACTION DISTANCE DIRECTION
S -> ACTION ANGLE ORIENTATION
S -> ACTION ANGLE ORIENTATION ADVERB ACTION
S -> ACTION OBJECT CONJ ACTION DIRECTION DISTANCE
S -> ACTION OBJECT DISTANCE CONJ ACTION DIRECTION DISTANCE
S -> ACTION OBJECT DISTANCE PREP VELOCITY
S -> ACTION OBJECT PREP COLOR CONJ ACTION DRAWING
S -> ACTION PREP PREP DIRECTION DISTANCE PREP VELOCITY ADVERB ACTION ANGLE UNIT ORIENTATION
S -> ACTION DIRECTION DISTANCE CONJ VELOCITY ADVERB ACTION ANGLE UNIT ORIENTATION
S -> ACTION CONJ ACTION DIRECTION PREP VELOCITY
S -> ACTION ANGLE UNIT ORIENTATION ADVERB ACTION DISTANCE PREP VELOCITY
S -> ACTION DIRECTION DISTANCE ADVERB ACTION DIRECTION DISTANCE
S -> ACTION DIRECTION DISTANCE UNIT PREP CONJ ITEM PREP VELOCITY UNIT ADVERB ACTION ANGLE UNIT ORIENTATION
S -> ACTION DIRECTION DISTANCE UNIT PREP VELOCITY UNIT CONJ ADVERB ACTION ANGLE UNIT ORIENTATION
S -> ACTION PREP OBJEC ITEM PREP COLOR CONJ ACTION DRAWING
S -> ACTION PREP COLOR CONJ ACTION DRAWING
S -> ACTION PREP CONJ DIRECTION DISTANCE UNIT PREP VELOCITY UNIT ADVERB ACTION PREP CONJ DISTANCE UNIT PREP VELOCITY UNIT
S -> ACTION ANGLE UNIT ORIENTATION
S -> ACTION OBJECT CONJ ACTION DISTANCE UNIT PREP VELOCITY UNIT
S -> ACTION DIRECTION DISTANCE UNIT PREP VELOCITY UNIT
S -> ACTION ANGLE UNIT ORIENTATION
S -> ACTION DIRECTION DISTANCE UNIT PREP VELOCITY UNIT ADVERB ACTION ANGLE UNIT ORIENTATION
S -> ACTION ANGLE UNIT ORIENTATION ADVERB ACTION DISTANCE UNIT PREP VELOCITY UNIT
S -> ACTION DRAWING CONJ ACTION PREP COLOR
S -> ACTION DIRECTION DISTANCE UNIT PREP CONJ ITEM PREP VELOCITY UNIT ADVERB ACTION ANGLE UNIT

ACTION -> 'move' | 'rotate' | 'draw' | 'changecolor' | 'pendown'
OBJECT -> 'the' ITEM
ITEM -> 'robot' | 'kinova_robot' | 'pen' | 'turtle' | 'color' |'pen' |'velocity'
PREP -> 'to' | 'from' | 'on' | 'with' | 'in' | 'off' | 'out' | 'into' | 'onto' | 'the' |'at' |'of'
DIRECTIO -> the DIRECTION
DIRECTION -> 'left' | 'right' | 'forward' | 'backward' | 'forwards' | 'backwards'
ORIENTATION -> 'clockwise' | 'counterclockwise'
VELOCITY -> DIGIT
SPEED -> '1' | '10' | '20'
ADVERB -> 'slowly' | 'quickly' | 'then'
CONJ -> 'and'|'a' |'at'|'the'
DRAWING -> 'a' FIGURE
FIGURE -> 'triangle' | 'square'
COLOR -> 'red' | 'blue' | 'green' | 'black' | 'white'
ANGLE -> {angles}
LOCATION -> 'the' PLACE
PLACE -> WORD
TOOL -> 'the' TOOL_ITEM
TOOL_ITEM -> WORD
PEN_DOWN -> 'pen_down'
NUMBER -> DIGIT | DIGIT NUMBER | NUMBER NUMBER
DIGIT -> {all_digits}
UNIT -> 'meters' | 'centimeters' | 'inches' | 'feet' | 'cm' | 'mm' | 'ft' | 'in' | 'm' | 'degrees' | 'ms' |'m/s'
DISTANCE -> DIGIT
"""


def parse_to_command_static_grammar(command, grammar_text, df = df_sbert):
    """
    Parses the preprocessed command into actions using a static CFG grammar.
    """
    # Preprocess the command to resolve verbs and clean the text
    resolved_command = preprocess_command(command, df_sbert)


    # Parse the resolved command into tokens
    tokens = resolved_command.split()
    #print(f'this is the tokens: <{tokens}>')

    # Load the grammar and initialize the parser
    cfg_grammar = CFG.fromstring(grammar_text)
    parser = ChartParser(cfg_grammar)

    actions = []  # Collect actions for the output format

    try:
        parse_trees = list(parser.parse(tokens))
        #print(f'this is the parse_trees {parse_trees}')
        if not parse_trees:
            return "No valid parse tree found."
        for tree in parse_trees:
            actions.extend(extract_actions(tree))

        # Format the actions in the expected YAML-like format
        output = "actions:\n"
        output += f"  number_of_actions: {len(actions)}\n\n"
        for i, action in enumerate(actions, 1):
            output += f"  action{i}:\n"
            for key, value in action.items():
                output += f"    {key}: {value}\n"
            output += "\n"

        return output.strip()

    except Exception as e:
        return f"Error parsing command: {str(e)}"






In [26]:
# Example usage
# df should be a DataFrame with verbs mapped to specific actions
command = "Pendown and move backward at 22"
print(parse_to_command_static_grammar(command, base_grammar, df_sbert))

actions:
  number_of_actions: 2

  action1:
    action_type: pen_down
    pen_down: True

  action2:
    action_type: move
    direction: backward
    distance: 1
    lin_velocity: 22


In [27]:



# Example commands
commands = [
    "Displace forward 2m at a velocity of 1.5m/s, then turn 45 degrees counterclockwise.",
    "Move backward 3m at 2m/s and then rotate 90 degrees counterclockwise",
"Sketch a square.",
"Changecolor to red and draw a triangle.",
"Turn 30 degrees counterclockwise.",
"Move backward 2m at 0.5m/s.",
"Spin 180 degrees counterclockwise.",
"Move right 0.5m at 0.8m/s, then rotate 60 degrees counterclockwise",
"Changecolor the pen to green and paint a triangle",
"Advance forward 4m at 1.2m/s, then turn 90 degrees counterclockwise",
"Rotate 45 degrees counterclockwise, then advance 2m at 0.9m/s.",
"Draw a square and changecolor to red",
"Shift forward 3m at a velocity of 2m/s then spin 90 degrees.",
"Pendown and move backward at 22",
"Move to the left 5 at 8 then rotate 62 degrees counterclockwise"
]

# Process each command
for command in commands:
    print(f"Command: {command}\n")
    print(parse_to_command_static_grammar(command, base_grammar, df_sbert))
    print("___________________________________________________________________________")

Command: Displace forward 2m at a velocity of 1.5m/s, then turn 45 degrees counterclockwise.

actions:
  number_of_actions: 2

  action1:
    action_type: move
    direction: forward
    distance: 2
    lin_velocity: 1.5

  action2:
    action_type: rotate
    orientation: counterclockwise
    angle: 45
___________________________________________________________________________
Command: Move backward 3m at 2m/s and then rotate 90 degrees counterclockwise

actions:
  number_of_actions: 2

  action1:
    action_type: move
    direction: backward
    distance: 3
    lin_velocity: 2

  action2:
    action_type: rotate
    orientation: counterclockwise
    angle: 90
___________________________________________________________________________
Command: Sketch a square.

actions:
  number_of_actions: 1

  action1:
    action_type: draw
    figure: square
___________________________________________________________________________
Command: Changecolor to red and draw a triangle.

actions:
  numbe

# LLM Based Method



In [28]:
import openai

# Replace this with your actual OpenAI API key

def generate_actions(command):
    """
    Sends the input command to GPT-3.5 and generates the actions in the desired format. If some values are not specified, just assume a baseline value of 1 for any parameter for which there is missing information.
    For every verb that is inputed, note that you can only resolve that verb to one of 5 options : rotate, move, change_color, draw, or pen_down.
    Any other verb would have to be first swtiched to the one it corresponds the most to in this list, then parsed in yaml.
    """
    prompt = f"""
    Command: "{command}"

    Parse the command into the following format:
    actions:
      number_of_actions: <number>

      action1:
        action_type: <action type>
        direction: <direction> = 'forward' (if not specified)
        distance: <distance> = 1 (if not specified)
        lin_velocity: <velocity> = 1 (if not specified)

      action2:
        action_type: <action type>
        direction: <direction>
        distance: <distance>
        lin_velocity: <velocity>

      # Extend as needed based on the number of actions in the command
    """

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that formats commands into structured action plans."},
                {"role": "user", "content": prompt},
            ],
            max_tokens=500,
            temperature=0.7,
        )

        # Extract the content of the response
        return response['choices'][0]['message']['content']

    except openai.error.OpenAIError as e:
        return f"An error occurred: {str(e)}"

# Example commands
commands = [
    "Displace forward 2m at a velocity of 1.5m/s, then turn 45 degrees counterclockwise.",
    "Move backward 3m at 2m/s and then rotate 90 degrees counterclockwise",
"Sketch a square.",
"Changecolor to red and draw a triangle.",
"Turn 30 degrees counterclockwise.",
"Move backward 2m at 0.5m/s.",
"Spin 180 degrees counterclockwise.",
"Move right 0.5m at 0.8m/s, then rotate 60 degrees counterclockwise",
"Changecolor the pen to green and paint a triangle",
"Advance forward 4m at 1.2m/s, then turn 90 degrees counterclockwise",
"Rotate 45 degrees counterclockwise, then advance 2m at 0.9m/s.",
"Draw a square and changecolor to red",
"Shift forward 3m at a velocity of 2m/s, then spin 90 degrees (assumed clockwise).",
"Pendown and move backward at 22",
"Move to the left 5 at 8, then rotate 62 degrees counterclockwise"
]

# Process each command
for command in commands:
    print(f"Command: {command}\n")
    actions = generate_actions(command)
    print(actions)
    print("\n" + "-"*50 + "\n")


Command: Displace forward 2m at a velocity of 1.5m/s, then turn 45 degrees counterclockwise.


    actions:
      number_of_actions: 2

      action1:
        action_type: "Displace"
        direction: "forward"
        distance: 2
        lin_velocity: 1.5

      action2:
        action_type: "Turn"
        direction: "counterclockwise"
        angle: 45

--------------------------------------------------

Command: Move backward 3m at 2m/s and then rotate 90 degrees counterclockwise


```yaml
actions:
  number_of_actions: 2

  action1:
    action_type: Move
    direction: backward
    distance: 3
    lin_velocity: 2

  action2:
    action_type: Rotate
    direction: counterclockwise
    angle: 90
```  

--------------------------------------------------

Command: Sketch a square.


```yaml
actions:
  number_of_actions: 4

  action1:
    action_type: "move"
    direction: "forward"
    distance: 1
    lin_velocity: 1

  action2:
    action_type: "rotate"
    direction: "right"
    angl